<div
  style="
    background-color: #f0f0f0;
    color:rgb(56, 56, 56);
    padding: 8px;
    display: flex;
    align-items: center;
    gap: 100px;
  "
>
  <img src="./images/brand.svg" style="max-height: 80px;">
  <strong>
    AI Saga: Data Science and Machine Learning</br>
    3.lab.2. Auto MPG Regression - Neural Networks
  </strong>
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from rich.table import Table
from rich.console import Console

al igual que la actividad pasada aqui vamos hacer priemro las configuraciones 

> nota pues solo explicare los mas importante 

- torch.manual_seed(42) fija las semillas aleatorias para resultados reproducibles
- np.random.seed(42) mismos pesos iniciales y división de datos en cada ejecución

### codificacion de las variables

- label_encoder = LabelEncoder()
- df["origin_encoded"] = label_encoder.fit_transform(df["origin"])


basicameente estas lienas convierten las variables a numero pues como y sabemos las redes neuronales solo leen numeros


### Variables 

- X = df[["cylinders", "displacement", "horsepower", "weight", "acceleration", "model_year", "origin_encoded"]].values
- y = df["mpg"].values

aqui simplemntes se seleccionan 7 caracteristicas para la entrada y en y se define la variable objetivo que para este caso sera MPG-Milla por galon


### Normalizacion: 

- scaler = StandardScaler()
- X_scaled = scaler.fit_transform(X)
- X_train, X_test, y_train, y_test = train_test_split


aqui son dos funciones las principales que son StandardScaler: Normaliza características (media=0, desviación=1) y train_test_split: que define 80% entrenamiento, 20% prueba


### Tensores: 


- X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
- y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

aqui basicamente se convierte a la forma en como fuciona la libreira PyT orch qu es [n] a [n, 1]



### Anaalisis

en resumen aqui preparamos los datos para el entrenamiento de las redes 



In [ ]:
torch.manual_seed(42)
np.random.seed(42)
# Cargar dataset desde seaborn
df = sns.load_dataset("mpg")
df = df.dropna()


label_encoder = LabelEncoder()
df["origin_encoded"] = label_encoder.fit_transform(df["origin"])

# Separar características y variable objetivo
X = df[
    [
        "cylinders",
        "displacement",
        "horsepower",
        "weight",
        "acceleration",
        "model_year",
        "origin_encoded",
    ]
].values
y = df["mpg"].values

## Preprocesamiento de Datos
# Normalizamos las características para mejorar el entrenamiento de la red neuronal
# La normalización es crucial para convergencia estable

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Conversión a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

##  Baselines establecidos de regresión polinomial


- mse_poly: Diccionario con errores cuadráticos medios de regresiones polinomiales previas
- r2_poly: Diccionario con coeficientes de determinación r2 correspondientes

en resumen estos son solo los resultados de la investigacion anteriror con regresion polinomila que usaremos para comparar mas adelante con el resultado de las redes neuronales


- best_mse_poly = min(mse_poly.values())
- best_r2_poly = max(r2_poly.values())

aqui se encuuntra el menor valor del mse y el mayor de r2 esto pues para saber quue valores se deben comparar despues del analisis



In [ ]:
mse_poly = {
    "Grado 1": 17.7,
    "Grado 2": 16.7,
    "Grado 3": 16.7,
    "Grado 4": 16.8,
    "Grado 5": 17.4,
}

r2_poly = {
    "Grado 1": 0.653,
    "Grado 2": 0.673,
    "Grado 3": 0.673,
    "Grado 4": 0.671,
    "Grado 5": 0.660,
}

# Mejor baseline anterior
best_mse_poly = min(mse_poly.values())
best_r2_poly = max(r2_poly.values())

 # Red neuroonal estandar

- MPGNet: Red neuronal para regresión de MPG
input_dim=7: 7 características de entrada cilindros, peso, etc.
hidden_dims=[64, 32, 16]: la arquitectura que va reduciendo neuronas 7→64→32→16→1
dropout_rate=0.2: 20% de neuronas se apagan para evitar sobreajuste


### construccion de las capas - ciclo for 


en el loop fotr vamos a econtrar alguan fucniones que ayudan a confirgurar la red  por lo qeue me enfocare en rekacionar eso y en el loop 

- Bucle: Crea capas dinámicamente según hidden_dims
- Linear: Conexiones  entre neuronas
- ReLU: Función de activación convierte negativos en 0
- Dropout: Regularización para prevenir memorización, es decir realiza una penalizacion para que el modelo se vea forzad oa encontrar mejores patrones 
- Salida: Una sola neurona valor de MPG predicho

### inicializacion de los pesos

- def _initialize_weights(self):


esta fucnion inicializa los pesoso con valores optimos para evitar gradientes que desaparescan 



en conclusion este modelo define una red neuronaala de aruqitectuura 7→64→32→16→1 con regularización  y inicialización inteligente para predecir MPG.


In [ ]:
class MPGNet(nn.Module):
    def __init__(self, input_dim=7, hidden_dims=[64, 32, 16], dropout_rate=0.2):
        super(MPGNet, self).__init__()

        layers = []
        prev_dim = input_dim

        # Capas ocultas
        for hidden_dim in hidden_dims:
            layers.extend(
                [nn.Linear(prev_dim, hidden_dim), nn.ReLU(), nn.Dropout(dropout_rate)]
            )
            prev_dim = hidden_dim

        # Capa de salida (regresión)
        layers.append(nn.Linear(prev_dim, 1))

        self.model = nn.Sequential(*layers)
        self._initialize_weights()

    def forward(self, x):
        return self.model(x)

    def _initialize_weights(self):
        for layer in self.model:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)

## Red neuronal simple 


- Shallow: Red  con solo 1 capa oculta
- Arquitectura: 7→16→1 oseaa mas sencilla 
- Menos dropout: 0.1 (10%) en comparacion del 20% del modelo etandaar


aqui este modelo nos servira para comparar si mas capas mejoran rendimiento y con menos sobreajuste puede  generalizar mejor con pocos datos


Define una red neuronal minima 7→16→1 para probar si la simplicidad puede competir con arquitecturas más complejas.

In [ ]:
class MPGNetShallow(nn.Module):
    def __init__(self, input_dim=7, hidden_dim=16, dropout_rate=0.1):
        super(MPGNetShallow, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 1),
        )

    def forward(self, x):
        return self.model(x)

 ## Red neuronal profunda


- Deep: Red profunda 7→128→64→32→16→1
- BatchNorm: estabiliza entrenamiento
- Más dropout: 30% para red más compleja
- 5 capas: Capacidad de aprender patrones complejos

esta red sirve paea probar si más complejidad mejora predicciones y Ver límites de lo que puede lograr una red grande

Definimos  una red neuronal muy profunda con una tecniica  (BatchNorm) para maximizar capacidad de aprendizaje y probar los límites del modelo.

In [ ]:
class MPGNetDeep(nn.Module):
    def __init__(self, input_dim=7, dropout_rate=0.3):
        super(MPGNetDeep, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        return self.model(x)

## Funcioon de entrenamiento 


- MSELoss: penaliza errores grandes
- Adam: optimizador adaptatif con weight_decay para regularizacion
- loss_history: Guarda perdida de cada época para gráficos


una vez se evalua no decolverra alguna  cosas en este caso son las siguientes 

- Historial de pérdida: Para gráficos de aprendizaje
- MSE train/test: detecta el sobre ajute y guardaa el error de tst vs el de train
- R2 train/test: Capacidad explicativa del modelo
- Predicciones: para gráficos scatter y análisi

en conclusion aqui tenmos la funcion de entrenamiento que funciona en cualquier modelo 


In [ ]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=500, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

    loss_history = []

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Backward pass
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

    # Evaluación final
    model.eval()
    with torch.no_grad():
        train_pred = model(X_train).numpy()
        test_pred = model(X_test).numpy()

    # Métricas
    mse_train = mean_squared_error(y_train.numpy(), train_pred)
    mse_test = mean_squared_error(y_test.numpy(), test_pred)
    r2_train = r2_score(y_train.numpy(), train_pred)
    r2_test = r2_score(y_test.numpy(), test_pred)

    return {
        "loss_history": loss_history,
        "mse_train": mse_train,
        "mse_test": mse_test,
        "r2_train": r2_train,
        "r2_test": r2_test,
        "predictions": test_pred,
    }

# Entrenamiento de los 3 modelos 


### Modelo Shallow

- menos épocas (300): red simple aprende más rápido
- learning rate alto (0.01): pasos grandes porque es menos compleja

### Modelo Standard

- 500 épocas: tiempo estándar para lleagr a una convergencia
- Learning rate medio (0.001): balance entre velocidad y estabilida

### # Modelo Deep


- Más épocas (700): Red compleja necesita más tiempo
- Learning rate bajo (0.0005): Pasos pequeños para estabilidad


en resumen aqui se entrenan lso  modelos y  laa pregunta aqui es ¿Por qué diferentes parámetros? es  por que las  redes más complejas requieren más tiempo y cuidado como ajustes específicos para cada arquitectura pues asi se optimiza el rendimiento



In [ ]:
# Modelo Shallow
model_shallow = MPGNetShallow()
results_shallow = train_model(
    model_shallow,
    X_train_tensor,
    y_train_tensor,
    X_test_tensor,
    y_test_tensor,
    epochs=300,
    lr=0.01,
)

# Modelo Standard
model_standard = MPGNet()
results_standard = train_model(
    model_standard,
    X_train_tensor,
    y_train_tensor,
    X_test_tensor,
    y_test_tensor,
    epochs=500,
    lr=0.001,
)

# Modelo Deep
model_deep = MPGNetDeep()
results_deep = train_model(
    model_deep,
    X_train_tensor,
    y_train_tensor,
    X_test_tensor,
    y_test_tensor,
    epochs=700,
    lr=0.0005,
)

## Analisis de resultados

> en este caso no enfatizare en el codigo si no en el outpout recibido


### Grafico 1 curvas de entrenameito 

- Shallow (azul): Converge rápidamente ~época 100, se estabiliza en MSE bajo
- Standard (naranja): Convergencia similar al shallow, curva suave
- Deep (verde): Converge más lentamente, necesita ~300 épocas


Redes simples aprenden más rápido que las complejas, en teoria todas convergen bien aunque deep si necesita alguito de tiempo


###  Scatter plot comparativos 


- eje X: MPG real del auto
- eje Y: MPG predicho por cada modelo
- linea roja diagonal: Predicción perfecta
- proximidad a la línea: Mejor precisión

entonces en base aa  los 3 graficos podemos definir 

- Shallow MSE: 8.54: puntos mas dispersos, mayor variabilidad
- Standard MSE: 6.38: puntos mas  cerca de la diagonal
- Deep MSE: 6.28: puntos mas concentrados cerca de la línea


en conclusion Todos superan significativamente el baseline polinomial definido (16.7), la mejora Deep vs Standard es pequeña y Standard ofrece el mejor balance entre precisión y simplicidad segun lso resultados

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Curvas de pérdida
axes[0, 0].plot(results_shallow["loss_history"], label="Shallow", alpha=0.7)
axes[0, 0].plot(results_standard["loss_history"], label="Standard", alpha=0.7)
axes[0, 0].plot(results_deep["loss_history"], label="Deep", alpha=0.7)
axes[0, 0].set_title("Curvas de Entrenamiento")
axes[0, 0].set_xlabel("Época")
axes[0, 0].set_ylabel("MSE Loss")
axes[0, 0].legend()
axes[0, 0].grid(True)

# Predicciones vs Reales - Shallow
axes[0, 1].scatter(y_test, results_shallow["predictions"], alpha=0.6, color="blue")
axes[0, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
axes[0, 1].set_title(f'Shallow Net (MSE: {results_shallow["mse_test"]:.2f})')
axes[0, 1].set_xlabel("MPG Real")
axes[0, 1].set_ylabel("MPG Predicho")

# Predicciones vs Reales - Standard
axes[1, 0].scatter(y_test, results_standard["predictions"], alpha=0.6, color="green")
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
axes[1, 0].set_title(f'Standard Net (MSE: {results_standard["mse_test"]:.2f})')
axes[1, 0].set_xlabel("MPG Real")
axes[1, 0].set_ylabel("MPG Predicho")

# Predicciones vs Reales - Deep
axes[1, 1].scatter(y_test, results_deep["predictions"], alpha=0.6, color="orange")
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
axes[1, 1].set_title(f'Deep Net (MSE: {results_deep["mse_test"]:.2f})')
axes[1, 1].set_xlabel("MPG Real")
axes[1, 1].set_ylabel("MPG Predicho")

plt.tight_layout()
plt.show()

## Tabla comparativa

ah9ora bien  aqui solo contruimos la tabaal coparativa para anlizar los resultados de lo modelo  y de lo que teniamos anteriormente por lo que en la tabla se definio (baseline) 

- 4 columnas con diferentes colores y alineaciones

- Primera parte: Agrega filas con resultados de regresión polinomial (baseline)
- Segunda parte: Agrega filas con resultados de redes neuronales
- Formato consistente: MSE con 1 decimal, R2 con 3 decimales


### Analisis


Las redes neuronales muestran una mejora respecto la regresión polinomial en la predicción de MPG, incluso con el modelo shallow que es el mas simple  superando al mejor polinomialMSE 16.7, R² 0.673 por un margen de 49% menos error y 20 puntos más de capacidad explicativa. , demostrando que las redes neuronales tienen una capacidad de modelado  mejor para capturar las relaciones complejas entre características automotrices del datacset  y eficiencia de combustible.

In [ ]:
results_table = Table(
    title="Comparación: Regresión Polinomial vs Redes Neuronales",
    show_header=True,
    header_style="bold",
)

results_table.add_column("Modelo", justify="left", style="cyan")
results_table.add_column("Arquitectura/Grado", justify="center", style="yellow")
results_table.add_column("MSE Test", justify="center", style="green")
results_table.add_column("R² Test", justify="center", style="blue")

# Resultados anteriores (regresión polinomial)
for grado, mse in mse_poly.items():
    r2 = r2_poly[grado]
    results_table.add_row("Regresión Polinomial", grado, f"{mse:.1f}", f"{r2:.3f}")

# Nuevos resultados (redes neuronales)
results_table.add_row(
    "Red Neuronal",
    "Shallow (7→16→1)",
    f'{results_shallow["mse_test"]:.1f}',
    f'{results_shallow["r2_test"]:.3f}',
)

results_table.add_row(
    "Red Neuronal",
    "Standard (7→64→32→16→1)",
    f'{results_standard["mse_test"]:.1f}',
    f'{results_standard["r2_test"]:.3f}',
)

results_table.add_row(
    "Red Neuronal",
    "Deep (7→128→64→32→16→1)",
    f'{results_deep["mse_test"]:.1f}',
    f'{results_deep["r2_test"]:.3f}',
)

console = Console()
console.print(results_table)

## Grafico de barra comparativo


### GRÁFICO MSE (Izquierda)

Barras rojas (polinomiales): Todas altas y similares 16.7-17.7
Barras azules (redes): Todas bajas con progresión descendente 8.5→6.4→6.3


### Grafico R2 (DERECHA)

Barras rojas : Todas bajas y agrupadas 0.65-0.67
Barras azules : Todas altas y progresiva 0.833→0.875→0.877


en conclusion los gráficos confirman visualmente la que las redesdes neuronales son superirores.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Preparar datos para gráficos
modelos = [
    "Poli Grado 1",
    "Poli Grado 2",
    "Poli Grado 3",
    "Poli Grado 4",
    "Poli Grado 5",
    "NN Shallow",
    "NN Standard",
    "NN Deep",
]

mse_valores = [
    17.7,
    16.7,
    16.7,
    16.8,
    17.4,
    results_shallow["mse_test"],
    results_standard["mse_test"],
    results_deep["mse_test"],
]

r2_valores = [
    0.653,
    0.673,
    0.673,
    0.671,
    0.660,
    results_shallow["r2_test"],
    results_standard["r2_test"],
    results_deep["r2_test"],
]

# Colores diferenciados
colors_mse = ["lightcoral"] * 5 + ["lightblue"] * 3
colors_r2 = ["lightcoral"] * 5 + ["lightblue"] * 3

# Gráfico MSE
bars1 = ax1.bar(modelos, mse_valores, color=colors_mse, alpha=0.7)
ax1.set_title("Error Cuadrático Medio (MSE)", fontsize=14, fontweight="bold")
ax1.set_ylabel("MSE")
ax1.tick_params(axis="x", rotation=45)
ax1.grid(True, alpha=0.3)

# Añadir valores en las barras
for bar, valor in zip(bars1, mse_valores):
    ax1.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.1,
        f"{valor:.1f}",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

# Gráfico R2
bars2 = ax2.bar(modelos, r2_valores, color=colors_r2, alpha=0.7)
ax2.set_title("Coeficiente de Determinación (R²)", fontsize=14, fontweight="bold")
ax2.set_ylabel("R²")
ax2.tick_params(axis="x", rotation=45)
ax2.grid(True, alpha=0.3)

# Añadir valores en las barras
for bar, valor in zip(bars2, r2_valores):
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.01,
        f"{valor:.3f}",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

In [ ]:
# Encontrar mejor modelo
all_models = [
    ("Polinomial Grado 2", best_mse_poly, best_r2_poly),
    ("NN Shallow", results_shallow["mse_test"], results_shallow["r2_test"]),
    ("NN Standard", results_standard["mse_test"], results_standard["r2_test"]),
    ("NN Deep", results_deep["mse_test"], results_deep["r2_test"]),
]

# Mejor por MSE menor es mejor
best_model_mse = min(all_models, key=lambda x: x[1])
# Mejor por R² mayor es mejor
best_model_r2 = max(all_models, key=lambda x: x[2])

print(f"MEJOR MODELO MSE: {best_model_mse[0]} - MSE: {best_model_mse[1]:.2f}")
print(f"MEJOR MODELO R2: {best_model_r2[0]} - R2: {best_model_r2[2]:.3f}")

en conclusion Se desarrolló una actividad de redes neuronales con PyTorch donde se implementaron dos modelos diferentes: una red neuronal de regresión con 3 capas ocultas que demostró mejor rendimiento que Ridge Regression en datos , y una red de clasificación binaria tipo XOR que logró alta precisión en la separación de clases no lineales. Los resultados mostraron que ambas redes funcionaron correctamente, aplicando técnicas de regularización como dropout y weight decay que permitieron obtener buenas métricas de evaluación (R2, F1-Score), confirmando que las arquitecturas neuronales pueden superar a modelos lineales en problemas complejos, tambien denotando que el emjor modelo fue el Deep.